## bronzeテーブルのパイプラインをDatabricks Auto Loaderにて作成

In [0]:
%run ./00_config

In [0]:
src_file_name = "temp-humidity_2025-05-17_09-27-51.csv"
src_file_path = f"{raw_data_volume_path}/{src_file_name}"

### CSVファイルをそのまま取り込むための、空テーブルを作成

In [0]:
# 用意したいカラム
schema = """
`Timestamp` STRING,
`Temperature` STRING,
`Humidity` STRING
"""

# Bronzeテーブルを作成
create_table_ddl = f"""
CREATE TABLE IF NOT EXISTS {bronze_table_path} (
    {schema},
    _rescued_data STRING,
    _datasource STRING,
    _ingest_timestamp timestamp
)
USING delta
"""

spark.sql(create_table_ddl)

In [0]:
# ソースからデータを読み込む
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "False")
    .load(src_file_path)
)

# 監査列として`_datasource`列と`_ingest_timestamp`列を追加
df = (
    df.select("*", "_metadata")
    .withColumn("_datasource", df["_metadata.file_path"])
    .withColumn("_ingest_timestamp", df["_metadata.file_modification_time"])
    .drop("_metadata")
)

In [0]:
df.display()

### 空テーブルにCSVファイルのデータを書き込み

In [0]:
(
    df.write.format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(bronze_table_path)
)


In [0]:
# データが書き込まれたことを確認
display(spark.table(bronze_table_path))

## 事後処理

In [0]:
# ストリーム処理を停止
for stream in spark.streams.active:
    stream.stop()